In [1]:
import torch
torch.cuda.is_available()

False

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from transformers import pipeline
from wordcloud import WordCloud
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

[]


<h3>Detecção de encoding</h3>
Leitura deste dataset retornou erro sem definição do encoding.
<pre><code>
import chardet
with open('DisneylandReviews.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read())
result
</code></pre>
Esta detecção levou vários minutos, mas foi necessária porque caracteres encontrados em seções avançadas do documento indicaram encoding 'latin-1' ou 'ISO-8859-1'.<br>
Fonte do dataset indicou, no entanto, o uso de encoding 'cp1252'.

In [3]:
df = pd.read_csv('DisneylandReviews.csv', encoding='cp1252')
print(df.shape)
print(df.nunique())
df.head()

(42656, 6)
Review_ID            42636
Rating                   5
Year_Month             112
Reviewer_Location      162
Review_Text          42632
Branch                   3
dtype: int64


,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong


In [4]:
df.drop_duplicates(subset='Review_Text', inplace=True, keep='first')

In [5]:
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def modelo_lexical(sentence):
    sentiment = sia.polarity_scores(sentence)
    print(sentiment)
    
model_neural = pipeline('sentiment-analysis', max_length=1024)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\rozan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [6]:
modelo_lexical("I don't feel very well")
modelo_lexical("I feel stoked about this!")

print(model_neural("I don't feel very well"))
print(model_neural("I feel stoked about this!"))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'neg': 0.404, 'neu': 0.596, 'pos': 0.0, 'compound': -0.2572}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
[{'label': 'NEGATIVE', 'score': 0.9997064471244812}]
[{'label': 'POSITIVE', 'score': 0.9998165965080261}]


In [7]:
resid = {}

for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['Review_Text']
        myid = row['Review_ID'] 
        resid[myid] = model_neural(text)
    except RuntimeError:
        print(f'Falha em id {myid}')

  0%|          | 0/42632 [00:00<?, ?it/s]

Falha em id 668777351
Falha em id 641689558
Falha em id 629073732
Falha em id 625303443
Falha em id 622768816
Falha em id 611414876
Falha em id 588779840
Falha em id 587107992
Falha em id 583615974
Falha em id 581160653
Falha em id 580315838
Falha em id 544141605
Falha em id 541903961
Falha em id 539556247
Falha em id 538740219
Falha em id 537126079
Falha em id 535957202
Falha em id 535385151
Falha em id 527224973
Falha em id 503964199
Falha em id 500960346
Falha em id 499017677
Falha em id 497042786
Falha em id 488612724
Falha em id 481108330
Falha em id 476680394
Falha em id 474451880
Falha em id 471360019
Falha em id 467718754
Falha em id 455237345
Falha em id 452236934
Falha em id 449804083
Falha em id 446283256
Falha em id 444393278
Falha em id 440832915
Falha em id 438193467
Falha em id 436586576
Falha em id 427559583
Falha em id 427242586
Falha em id 421737079
Falha em id 411130071
Falha em id 406843321
Falha em id 398996871
Falha em id 394152992
Falha em id 387558771
Falha em i

In [9]:
import csv

with open('resultado.csv', mode='w', newline='') as f:
    writer = csv.writer(f)
    for key, value in resid.items():
        writer.writerow([key, value])

In [ ]:
resultado = pd.DataFrame(resid).T
resultado = resultado.reset_index().rename(columns={"index": 'Review_ID', 0: 'Label'})
resultado.head()

In [ ]:
resultado[['Sentiment', 'Score']] = resultado['Label'].apply(lambda x: pd.Series([x['label'], x['score']]))
resultado.drop('Label', axis=1, inplace=True)
resultado.head()

<h1>Nuvem de palavras</h1>

In [ ]:
df['Review_Text']=df['Review_Text'].astype('str')
review_text = " ".join(i for i in df['Review_Text'])

In [ ]:
# todas as avaliações
wc = WordCloud(max_font_size=50, max_words=100, background_color="white", colormap='Spectral').generate(review_text)
plt.figure(figsize=(10,10))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# avaliações positivas
df_positif = df.loc[df['Sentiment'] == "POSITIVE"]
review_text = " ".join(i for i in df_positif['Review_Text'])
wc = WordCloud(max_font_size=50, max_words=100, background_color="white", colormap='Blues', ).generate(review_text)
plt.figure(figsize=(10,10))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# avaliações negativas
df_positif = df.loc[df['Sentiment'] == "NEGATIVE"]
review_text = " ".join(i for i in df_positif['Review_Text'])
wc = WordCloud(max_font_size=50, max_words=100, background_color="white", colormap='Reds', ).generate(review_text)
plt.figure(figsize=(10,10))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()